In [1]:
import os
import datetime
import random
import csv

import PIL
from PIL import Image
from PIL import ImageEnhance

import tensorflow as tf
from tensorflow.contrib.layers import fully_connected
from tensorflow.contrib.layers import dropout
from tensorflow.contrib.layers import conv2d
from tensorflow.contrib.layers import variance_scaling_initializer
from tensorflow.contrib.layers import batch_norm

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# path to dataset 
DATA_ROOT = r"/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge"
assert(os.path.exists(DATA_ROOT))

TRAIN_IMAGES = os.path.join(DATA_ROOT, "train")
assert(os.path.exists(TRAIN_IMAGES))

TRAIN_MASKS = os.path.join(DATA_ROOT, "train_masks")
assert(os.path.exists(TRAIN_MASKS))

TRAIN_MASKS_CSV = os.path.join(DATA_ROOT, "train_masks.csv")
assert(os.path.exists(TRAIN_MASKS_CSV))

METADATA_CSV = os.path.join(DATA_ROOT, "metadata.csv")
assert(os.path.exists(METADATA_CSV))

TEST_IMAGES = os.path.join(DATA_ROOT, "test")
assert(os.path.exists(TEST_IMAGES))

In [2]:
def _labels_files(filename, labels_dir):
    separator = "/"
    basename = tf.string_split([filename], separator).values[-1]

    name = tf.string_split([basename], ".").values[0]

    new_name = tf.string_join([name, "mask.gif"], "_")

    lables_file = tf.string_join([labels_dir, new_name], separator)

    return lables_file


files = tf.train.match_filenames_once([TRAIN_IMAGES+"/*"] * 2)

# files = tf.random_shuffle(files)

# print(TRAIN_IMAGES+"/*")
# 
labels = tf.map_fn(lambda x: _labels_files(x, TRAIN_MASKS), files)
# labels = tf.string_split(files, "/")

# test = tf.string_split(["test test"], " ")
# test_values = test.values

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()
tf.local_variables_initializer().run()

# print(test_values.eval())

print(files.eval()[:5])
# print(len(files.eval()))

print(labels.eval()[:5])

features_dataset = files.eval()
labels_dataset = labels.eval()

for index, file in enumerate(features_dataset):
    basename = os.path.splitext(os.path.basename(file))[0]
    
    assert(basename in labels_dataset[index])
# print(len(labels.eval()).value)
print("finish")
sess.close()

[ b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train/4e7bc95552ed_06.jpg'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train/65051cfe0789_04.jpg'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train/784ca55262c2_06.jpg'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train/4f0397cf7937_02.jpg'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train/8d5423cb763c_05.jpg']
[ b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train_masks/4e7bc95552ed_06_mask.gif'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENCE/KAGGLE_COMPETITIONS/Carvana_Image_Masking_Challenge/train_masks/65051cfe0789_04_mask.gif'
 b'/run/media/onionhuang/HDD/ARTIFICIAL_INTELLIGENC

In [3]:
FLAGS = tf.app.flags.FLAGS

# tf.app.flags.DEFINE_integer("batch_size", 10, """batch size""")

# path to dataset
# DATA_ROOT = os.path.join("/run/media/onionhuang/HDD/artificial_intelligence",
#                          "kaggle/Carvana-Image-Masking-Challenge")

# assert (os.path.exists(DATA_ROOT))

# tf.app.flags.DEFINE_string("TRAIN_IMAGES",
#                            os.path.join(DATA_ROOT, "train"),
#                            """path to training images""")

# tf.app.flags.DEFINE_string("TRAIN_MASKS",
#                            os.path.join(DATA_ROOT, "train_masks"),
#                            """path to training images masks""")

# tf.app.flags.DEFINE_string("TRAIN_MASKS_CSV",
#                            os.path.join(DATA_ROOT, "train_masks.csv"),
#                            """path to training images masks in csv format""")

# tf.app.flags.DEFINE_string("METADATA_CSV",
#                            os.path.join(DATA_ROOT, "metadata.csv"),
#                            """path to images metadata in csv format""")

# tf.app.flags.DEFINE_string("TEST_IMAGES",
#                            os.path.join(DATA_ROOT, "test"),
#                            """path to test images""")

SOURCE_IMAGE_HEIGHT = 1280

SOURCE_IMAGE_WIDTH = 1918

TARGET_IMAGE_SIZE = 224

In [4]:
def read_inputs():
    if not TRAIN_IMAGES:
        raise ValueError("TRAIN_IMAGES is empty")

#     filepath = [
#         os.path.join(TRAIN_IMAGES, i)
#         for i in os.listdir(TRAIN_IMAGES)[:10]
#     ]
    
    filepath = [
        os.path.join(TRAIN_IMAGES, "00087a6bd4dc_01.jpg")
    ]

    for f in filepath:
        if not tf.gfile.Exists(f):
            raise ValueError("file does not exists:", f)

    file_queue = tf.train.string_input_producer(filepath)

    image_reader = tf.WholeFileReader()

    _, image_file = image_reader.read(file_queue)

    image = tf.image.decode_image(image_file)
    
    image.set_shape((SOURCE_IMAGE_HEIGHT, SOURCE_IMAGE_WIDTH, 3))
    
    return image

In [5]:
def batch_inputs():
    pass

In [6]:
def read_labels():
    if not TRAIN_IMAGES:
        raise ValueError("TRAIN_IMAGES is empty")
        
    if not TRAIN_MASKS:
        raise ValueError("TRAIN_MASKS is empty")

    filepath = [
        os.path.join(
            TRAIN_MASKS, 
            os.path.splitext(i)[0]+"_mask.gif",
        )
        for i in ["00087a6bd4dc_01.jpg"]
    ]

    for f in filepath:
        if not tf.gfile.Exists(f):
            raise ValueError("file does not exists:", f)

    file_queue = tf.train.string_input_producer(filepath)

    image_reader = tf.WholeFileReader()

    _, image_file = image_reader.read(file_queue)

    image = tf.image.decode_image(image_file)
    
    image = tf.gather(image, 0)
    
    image = tf.image.rgb_to_grayscale(image)
    
    image.set_shape((SOURCE_IMAGE_HEIGHT, SOURCE_IMAGE_WIDTH, 1))
    
    return image

In [7]:
def batch_labels():
    pass

In [8]:
def resize_pad(
    inputs, 
    resize=True, 
    resize_shape=(224, 224), 
    crop_pad=False, 
    crop_pad_shape=(224, 224),
#     random_flip=False,
#     random_adjustment=False,
#     standardize=True,
):
    
    image = inputs
    
    if resize:
        image = tf.image.resize_images(
            inputs, 
            resize_shape, 
#             tf.image.ResizeMethod.BICUBIC,
            align_corners=True,
        )
        
    if crop_pad:
        image = tf.image.resize_image_with_crop_or_pad(
            image, 
            crop_pad_shape[0], 
            crop_pad_shape[1])
        
#     if random_flip:
#         image = random_flip(image)
        
#     if random_adjustment:
#         image = random_adjustment(image)
        
#     if standardize:
#         image = tf.image.per_image_standardization(image)
        
    return image

In [9]:
def random_adjustment(
    inputs,
    brightness=True,
    brightness_delta=0.75,
    contrast=True,
    contrast_lower=0.25,
    contrast_upper=1.5,
    hue=False,
    hue_delta=0.5,
    saturation=True,
    saturation_lower=0.0,
    saturation_upper=1.25,
):
    
    image = inputs
    
    if brightness:
        image = tf.image.random_brightness(image, max_delta=brightness_delta)
    
    if contrast:
        image = tf.image.random_contrast(image, lower=contrast_lower, upper=contrast_upper)
    
    if hue:
        image = tf.image.random_hue(image, hue_delta)
    
    if saturation:
        image = tf.image.random_saturation(image, lower=saturation_lower, upper=saturation_upper)
    
    return image

In [10]:
def random_flip(
    inputs,
    vertical=False,
    horizontal=True,
):
    
    image = inputs
    
    if vertical:
        image = tf.image.random_flip_up_down(image)
        
    if horizontal:
        image = tf.image.random_flip_left_right(image)
        
    return image

In [11]:
def standardize(inputs):
    
    image = inputs
    
#     image = tf.image.convert_image_dtype(image, tf.float32)
    
#     image_max = tf.reduce_max(tf.reshape(image, [-1]))
    
#     image = image / image_max
    
    image = tf.image.per_image_standardization(image)
    
    return image

In [12]:
import pandas as pd

csv = pd.read_csv(TRAIN_MASKS_CSV, header=0)

target = csv.loc[0].rle_mask

print(csv.loc[0].rle_mask)

879386 40 881253 141 883140 205 885009 17 885032 259 886923 308 888839 328 890754 340 892670 347 894587 352 896503 357 898420 360 900336 364 902253 367 904170 370 906086 374 908003 377 909920 379 911837 381 913754 384 915671 386 917588 388 919505 391 921422 393 923339 395 925256 397 927173 399 929090 401 931007 403 932925 404 934842 406 936759 408 938676 410 940593 412 942510 414 944428 415 946345 416 948262 418 950179 420 952097 421 954014 423 955931 425 957848 426 959766 427 961683 429 963600 431 965517 432 967435 433 969352 435 971269 437 973186 438 975104 439 977021 441 978938 442 980855 444 982773 445 984690 447 986607 448 988525 449 990442 451 992359 452 994277 453 996194 455 998111 456 1000028 458 1001946 459 1003863 460 1005780 462 1007698 463 1009615 464 1011532 466 1013450 467 1015367 468 1017284 470 1019202 471 1021119 472 1023036 474 1024954 475 1026871 476 1028788 478 1030706 479 1032623 480 1034540 482 1036458 482 1038375 484 1040292 486 1042210 486 1044127 488 1046044 49

In [13]:
def rle_encode(img):
    '''
    img: numpy array, 1 - mask, 0 - background
    Returns run length as string formated
    '''
    
    pixels = img.flatten()
    pixels[0] = 0
    pixels[-1] = 0
    runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
    runs[1::2] = runs[1::2] - runs[:-1:2]
    return " ".join(str(x) for x in runs)

In [14]:
# def rle_encode(mask_image):
#     pixels = mask_image.flatten()
#     # We avoid issues with '1' at the start or end (at the corners of 
#     # the original image) by setting those pixels to '0' explicitly.
#     # We do not expect these to be non-zero for an accurate mask, 
#     # so this should not harm the score.
#     pixels[0] = 0
#     pixels[-1] = 0
#     runs = np.where(pixels[1:] != pixels[:-1])[0] + 2
#     runs[1::2] = runs[1::2] - runs[:-1:2]
#     return runs


# def rle_to_string(runs):
#     return ' '.join(str(x) for x in runs)

In [15]:
image = read_inputs()
label = read_labels()

# label_max = tf.reduce_max(label)

# new_width = FLAGS.TARGET_IMAGE_SIZE
# new_height = int(
#     float(FLAGS.SOURCE_IMAGE_HEIGHT) / (
#         float(FLAGS.SOURCE_IMAGE_WIDTH) / float(FLAGS.TARGET_IMAGE_SIZE)
#     )
# )

# image = random_adjustment(image, hue=True)

# image = resize_pad(
#     image, 
#     resize=True, 
#     resize_shape=(new_height, new_width), 
#     crop_pad=False, 
#     crop_pad_shape=(
#         FLAGS.TARGET_IMAGE_SIZE, 
#         FLAGS.TARGET_IMAGE_SIZE,
#     ),
# #     random_flip=False,
# #     random_adjustment=True,
# #     standardize=False,
# )
# image = tf.image.convert_image_dtype(image, tf.float32)
# image = tf.image.adjust_gamma(image, gamma=4.0)

# image = tf.image.adjust_contrast(image, contrast_factor=4)
# label_f = tf.contrib.layers.flatten(label)
# label_f = tf.reshape(label, [-1])

# image = standardize(image)

# image_batch = tf.train.shuffle_batch(
#     [image],
#     batch_size=3,
#     num_threads=2,
#     capacity=50,
#     min_after_dequeue=1)

sess = tf.InteractiveSession()

tf.global_variables_initializer().run()
tf.local_variables_initializer().run()

coord = tf.train.Coordinator()
threads = tf.train.start_queue_runners(coord=coord)

for i in range(1):
    
    img = label.eval()
    
    src = rle_encode(img)
    
    if src != target:
        print("different")
    
#     for hi, h in enumerate(img):
#         for wi, w in enumerate(h):
            
#     print(label.eval())
#     print(label.eval())
#     plt.imshow(label.eval())
#     plt.imshow(image.eval().astype(np.uint8))
#     plt.show()

coord.request_stop()
coord.join(threads)

sess.close()


In [16]:
x = tf.placeholder(tf.string, [None])

sess = tf.InteractiveSession()

print(x.eval(feed_dict={
    x: ["a", "b"]
}))
sess.close()

['a' 'b']
